In [ ]:
from vista_ssm import vista_ssm_Funcs as vista
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi']=600
import numpy as np
import pandas as pd
import folium

In [2]:
M=np.load('../data/Euro_data.npy')
M=M[[x for x in range(55) if ((x != 50)&(x != 19))]]
M=M[:,:,:174] 
Md=M[:,:,1:]-M[:,:,0:-1]

dataset=Md
dataset=dataset/np.kron(np.kron(np.ones((53,1)),dataset.max(axis=0).max(axis=1)).reshape(53,2,1),np.ones(173))

dataset=np.transpose(dataset,(0,2,1))
dataset=dataset.reshape(dataset.shape[0],dataset.shape[1],dataset.shape[2],1)
print(dataset.shape)

(53, 173, 2, 1)


In [ ]:
for nc in range(3,9):
    for dx in range(2,9):
        param_dic={'DIM_X': dx,
           'DIM_Y': dataset.shape[2],
           'NUM_DATA': dataset.shape[0],
           'N_CLUSTER' : nc,
           'NUM_CPU' : 30,
           'FIX' : [],
           'NUM_LGSSM' : 30,
           'MAX_ITER' : 1000,
           'EPSILON' : 0.1,
           'BIC' : True}
        
        result=vista.runVISTA('ident',param_dic,dataset)
        vista.savedic(result,'../results/epidemiological/epidemiological_ident_'+str(nc)+'_cluster_'+str(dx)+'_latent_0.pickle')

In [3]:
vista.agg_perf('../results/epidemiological/epidemiological_ident_',[3,4,5,6,7,8],list(range(2,9)),1,criteria=[1],std=[False,True])

Information Criteria:

ABIC:
           3          4          5          6          7          8
2  -9.42E+04  -9.73E+04  -9.84E+04  -9.85E+04  -9.86E+04  -9.97E+04
3  -9.73E+04  -1.00E+05  -1.02E+05  -1.02E+05  -1.01E+05  -1.02E+05
4  -9.79E+04  -1.04E+05  -1.03E+05  -1.04E+05  -1.01E+05  -1.02E+05
5  -9.92E+04  -9.49E+04  -1.02E+05  -1.04E+05  -1.05E+05  -1.05E+05
6  -9.51E+04  -1.02E+05  -1.02E+05  -1.03E+05  -1.05E+05  -1.04E+05
7  -9.90E+04  -9.36E+04  -9.95E+04  -1.04E+05  -9.82E+04  -1.02E+05
8  -9.87E+04  -9.54E+04  -9.81E+04  -1.03E+05  -1.01E+05  -1.00E+05


In [4]:
#https://realpython.com/python-folium-web-maps-from-data/

def countryplot(s,color='Set1'):
    result=vista.loaddic(s)
    
    list_countries=np.load('../data/country_list.npy')
    list_countries=np.delete(list_countries,50)
    list_countries[26]='Kosovo'
    list_countries[34]='Netherlands'
    list_countries[39]='Moldova'
    list_countries[41]='Russia'
    list_countries[50]='Turkey'
    list_countries[52]='United Kingdom'
    list_countries[11]='Czech Rep.'
    list_countries[7]='Bosnia and Herz.'
    list_countries[35]='Macedonia'
    list_countries=np.delete(list_countries,19)
    
    df=pd.DataFrame()
    df["Country/region"]=list_countries
    df['Class']=result['label']
    for i in range(df['Class'].max()+1):
        lc=list(df[df['Class']==i]['Country/region'].to_numpy())
        print('Cluster '+str(i)+": ", end='')
        for l in range(len(lc)):
            if l<len(lc)-1:
                print(f'{lc[l]}', end=', ')
            else:
                print(f'{lc[l]}', end='; ')
        print('')

    political_countries_url = (
        "http://geojson.xyz/naturalearth-3.3.0/ne_50m_admin_0_countries.geojson"
    )
    
    m = folium.Map(location=(45, -15), zoom_start=3, tiles="cartodb positron no labels")
    folium.Choropleth(
        geo_data=political_countries_url,
        data=df,
        columns=("Country/region","Class"),
        key_on="feature.properties.name",
        fill_color=color, #brewer palette
        bins=list(range(len(np.unique(result['label']))+1)),
        fill_opacity=0.8,
        line_opacity=0.3,
        nan_fill_color="white",
        legend_name="Cluster",
    ).add_to(m)
    #m.fit_bounds([[30,10], [70,55]]) 
    m.save("map.html")
    display(m)

In [ ]:
for nc in range(3,9):
    for dx in range(2,9):
        print((nc,dx))
        countryplot('../results/epidemiological/epidemiological_ident_'+str(nc)+'_cluster_'+str(dx)+'_latent_0.pickle')